In [ ]:
# @title
# Imports utilities and packages
import warnings
import pandas as pd
from datetime import datetime
import os

# Telling function to ignore all warnings
warnings.filterwarnings('ignore')
pd.set_option('display.width', 1000)

def dataReport_to_csv(filename):

  # Check if the filename contains


  # Import file from side column while preserving data
  # Modify this to read CSVs in case clients send us CSV
  df = pd.read_excel(filename, 'Request Shipping Labels', header=None)

  # Remove unnecessary columns after 'REQUEST SHIPPING LABELS'
  df = df.loc[:, :11]
  print(df.head())
  df = df.drop(df.columns[[4, 6, 7, 8]], axis=1)
  new_header = df.iloc[0]
  df = df[1:]
  df.columns = new_header



  # Remove the summary lines from the file.
  rem = "SUMMARY"
  df = df[~df['PO NUM'].str.contains(rem)]

  ##############################################################################
  # Remove Nan from the carton number and replace with 6 JUST FOR TESTING
  df['SHIP CARTON NUMBER'].fillna('6', inplace=True)
  ##############################################################################

  # Add placeholder for all the BOL/Tracking field
  df['ship_info.ship_info_tracking'] = "PLACEHOLDER"

  # Set the headers to the right CSV mapping
  df.rename(columns={
      'VENDOR':       'vendor.tp_directory',
      'RETAILER':     'retailer.tp_name',
      'SHIP TO LOC':    'ship_info.ship_to_location.tp_location_code',
      'PO NUM':    'po.po_num',
      'MARK FOR LOC':    'po.mark_for_location.tp_location_code',
      'GTIN/UPC':   'po.ship_carton.po_item_pack.po_item.product.product_gtin',
      'QTY SHIPPED':   'po.ship_carton.po_item_pack.po_item_pack_qty',
      'SHIP CARTON NUMBER':   'po.ship_carton.ship_carton_number'
    }, inplace=True)

  # Create file name variables for the format: DATE_PONUM_VENDOR_RETAILER.CSV
  file_name_breaks = [
      datetime.today().strftime('%Y-%m-%d'),
      # df[3].values[0],
      # df[0].values[0].title(),
      # df[1].values[0].title()
    ]
  file_name = ''
  for i in file_name_breaks:
    file_name = file_name + i + "_"
  file_name = "CSV__Report_" + file_name[:-1] + ".csv"
  print(df.head())

  # Spit out the CSV file
  with open(file_name, 'w') as csv_file:
      df.to_csv(index=False, path_or_buf=csv_file)

'''

  Use cases to check:
  1. Whether the file provided is an Excel file or the CSV file
  2. Make sure the script doesn't run the CSV's that it produced
    -- Solution: Since I append CSV__ in fron the file which is kindof
    like a unique identifier.
    Look for the first five characters for the filename so that
    it tells you that the file was generated

'''


# @title
directory_path = '/content/'
directory_files = os.listdir(directory_path)
files_to_process = []
for f in directory_files:
  if f[-4:] == "xlsx" or f[-3:] == "csv" and f[0:5] != "CSV__":
    files_to_process.append(f)

for f in files_to_process:
  dataReport_to_csv(f)

print("Finished processing")

       0                     1            2        3            4             5               6      7     8             9            10                  11
0  VENDOR              RETAILER  SHIP TO LOC   PO NUM  RELEASE NUM  MARK FOR LOC  VENDOR_ITM_NUM  COLOR  SIZE      GTIN/UPC  QTY SHIPPED  SHIP CARTON NUMBER
1   nancy  Bloomingdales Outlet           CI  2949358      2949358          0191             NaN    NaN   NaN  492037570708            1            Carton 1
2   nancy  Bloomingdales Outlet           CI  2949358      2949358          0191             NaN    NaN   NaN  492037570715            3                 NaN
3   nancy  Bloomingdales Outlet           CI  2949358      2949358          0191             NaN    NaN   NaN  492037570722            3                 NaN
4   nancy  Bloomingdales Outlet           CI  2949358      2949358          0191             NaN    NaN   NaN  492037570739            2                 NaN
0 vendor.tp_directory      retailer.tp_name ship_info.ship